In [15]:

text = '''Next character prediction is a fundamental task in the field of natural language processing (NLP) that involves predicting the next character in a sequence of text based on the characters that precede it. This task is essential for various applications, including text auto-completion, spell checking, and even in the development of sophisticated AI models capable of generating human-like text.

At its core, next character prediction relies on statistical models or deep learning algorithms to analyze a given sequence of text and predict which character is most likely to follow. These predictions are based on patterns and relationships learned from large datasets of text during the training phase of the model.

One of the most popular approaches to next character prediction involves the use of Recurrent Neural Networks (RNNs), and more specifically, a variant called Long Short-Term Memory (LSTM) networks. RNNs are particularly well-suited for sequential data like text, as they can maintain information in 'memory' about previous characters to inform the prediction of the next character. LSTM networks enhance this capability by being able to remember long-term dependencies, making them even more effective for next character prediction tasks.

Training a model for next character prediction involves feeding it large amounts of text data, allowing it to learn the probability of each character's appearance following a sequence of characters. During this training process, the model adjusts its parameters to minimize the difference between its predictions and the actual outcomes, thus improving its predictive accuracy over time.

Once trained, the model can be used to predict the next character in a given piece of text by considering the sequence of characters that precede it. This can enhance user experience in text editing software, improve efficiency in coding environments with auto-completion features, and enable more natural interactions with AI-based chatbots and virtual assistants.

In summary, next character prediction plays a crucial role in enhancing the capabilities of various NLP applications, making text-based interactions more efficient, accurate, and human-like. Through the use of advanced machine learning models like RNNs and LSTMs, next character prediction continues to evolve, opening new possibilities for the future of text-based technology.'''

In [16]:
import torch
import time
import torch.nn as nn
import torch.optim as optim
import numpy as np
from sklearn.model_selection import train_test_split
from torchsummary import summary

In [17]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [18]:
import torch
from sklearn.model_selection import train_test_split

def preprocess_data(text, max_length):
    sequences = [text[i:i + max_length] for i in range(len(text) - max_length)]
    labels = [text[i + max_length] for i in range(len(text) - max_length)]

    # Creating character vocabulary
    chars = sorted(list(set(text)))
    char_to_ix = {ch: i for i, ch in enumerate(chars)}

    X = torch.tensor([[char_to_ix[ch] for ch in seq] for seq in sequences], dtype=torch.long)
    y = torch.tensor([char_to_ix[label] for label in labels], dtype=torch.long)

    # Splitting the dataset into training and validation sets
    X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=100)

    return X_train, y_train, X_val, y_val, len(chars)

max_length = 10
X_train, y_train, X_val, y_val, input_size = preprocess_data(text, max_length)

X_train = X_train.to(device)
y_train = y_train.to(device)
X_val = X_val.to(device)
y_val = y_val.to(device)

In [19]:
class TransformerModel(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, num_layers, nhead):
        super(TransformerModel, self).__init__()
        self.embedding = nn.Embedding(input_size, hidden_size)
        encoder_layers = nn.TransformerEncoderLayer(hidden_size, nhead)
        self.transformer_encoder = nn.TransformerEncoder(encoder_layers, num_layers)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        embedded = self.embedding(x)
        transformer_output = self.transformer_encoder(embedded)
        output = self.fc(transformer_output[:, -1, :])  # Get the output of the last Transformer block
        return output


In [20]:
hidden_size = 128
num_layers = 3
num_head = 2
learning_rate = 0.001
epochs = 100


In [21]:
model = TransformerModel(input_size, hidden_size, input_size, num_layers, num_head)
model = model.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

/usr/local/lib/python3.10/dist-packages/torch/nn/modules/transformer.py:286: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")


In [22]:
start  = time.time()
for epoch in range(epochs):
    model.train()
    optimizer.zero_grad()

    output = model(X_train)
    loss = criterion(output, y_train)
    loss.backward()
    optimizer.step()

    model.eval()
    with torch.no_grad():
        val_output = model(X_val)
        val_loss = criterion(val_output, y_val)
        _, predicted = torch.max(val_output, 1)
        val_accuracy = (predicted == y_val).float().mean()

    if (epoch+1) % 10 == 0:
        print(f'Epoch {epoch+1}, train Loss: {loss.item()}, val loss: {val_loss.item()}, val accuracy: {val_accuracy.item()}')
end = time.time()
print("Total_time ", end-start )

Epoch 10, train Loss: 2.6195342540740967, val loss: 2.660613536834717, val accuracy: 0.25995805859565735
Epoch 20, train Loss: 2.411149263381958, val loss: 2.5072903633117676, val accuracy: 0.2557651996612549
Epoch 30, train Loss: 2.2944910526275635, val loss: 2.4634010791778564, val accuracy: 0.2620545029640198
Epoch 40, train Loss: 2.2326853275299072, val loss: 2.4134485721588135, val accuracy: 0.27463310956954956
Epoch 50, train Loss: 2.210275888442993, val loss: 2.4017648696899414, val accuracy: 0.2620545029640198
Epoch 60, train Loss: 2.1934540271759033, val loss: 2.4023497104644775, val accuracy: 0.2704402506351471
Epoch 70, train Loss: 2.1836631298065186, val loss: 2.4033145904541016, val accuracy: 0.2704402506351471
Epoch 80, train Loss: 2.18159556388855, val loss: 2.4052844047546387, val accuracy: 0.2641509473323822
Epoch 90, train Loss: 2.1790931224823, val loss: 2.4079995155334473, val accuracy: 0.26624736189842224
Epoch 100, train Loss: 2.1673896312713623, val loss: 2.40906

In [23]:
max_length = 20
X_train, y_train, X_val, y_val, char_len = preprocess_data(text, max_length)

X_train = X_train.to(device)
y_train = y_train.to(device)
X_val = X_val.to(device)
y_val = y_val.to(device)

hidden_size = 128
num_layers = 3
num_head = 2
learning_rate = 0.001
epochs = 100
model = TransformerModel(char_len, hidden_size, char_len, num_layers, num_head)
model = model.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

start = time.time()
for epoch in range(epochs):
    model.train()
    optimizer.zero_grad()

    output = model(X_train)
    loss = criterion(output, y_train)
    loss.backward()
    optimizer.step()

    model.eval()
    with torch.no_grad():
        val_output = model(X_val)
        val_loss = criterion(val_output, y_val)
        _, predicted = torch.max(val_output, 1)
        val_accuracy = (predicted == y_val).float().mean()

    if (epoch+1) % 10 == 0:
        print(f'Epoch {epoch+1}, train Loss: {loss.item()}, val loss: {val_loss.item()}, val accuracy: {val_accuracy.item()}')
end = time.time()
print("Total time :", end-start)

/usr/local/lib/python3.10/dist-packages/torch/nn/modules/transformer.py:286: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")


Epoch 10, train Loss: 2.5927019119262695, val loss: 2.5438458919525146, val accuracy: 0.22947368025779724
Epoch 20, train Loss: 2.3938372135162354, val loss: 2.4202120304107666, val accuracy: 0.2631578743457794
Epoch 30, train Loss: 2.291459321975708, val loss: 2.385174512863159, val accuracy: 0.29894736409187317
Epoch 40, train Loss: 2.253955364227295, val loss: 2.350126266479492, val accuracy: 0.27368420362472534
Epoch 50, train Loss: 2.2231285572052, val loss: 2.3384757041931152, val accuracy: 0.2800000011920929
Epoch 60, train Loss: 2.2108373641967773, val loss: 2.334221124649048, val accuracy: 0.2673684060573578
Epoch 70, train Loss: 2.2057182788848877, val loss: 2.328277111053467, val accuracy: 0.3115789294242859
Epoch 80, train Loss: 2.2011234760284424, val loss: 2.3347439765930176, val accuracy: 0.3136841952800751
Epoch 90, train Loss: 2.19429087638855, val loss: 2.3314781188964844, val accuracy: 0.30315786600112915
Epoch 100, train Loss: 2.188854455947876, val loss: 2.33175182

In [24]:
max_length = 30
X_train, y_train, X_val, y_val, char3 = preprocess_data(text, max_length)

X_train = X_train.to(device)
y_train = y_train.to(device)
X_val = X_val.to(device)
y_val = y_val.to(device)

hidden_size = 128
num_layers = 3
num_head = 2
learning_rate = 0.001
epochs = 100
model = TransformerModel(char3, hidden_size, char3, num_layers, num_head)
model = model.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

start = time.time()
for epoch in range(epochs):
    model.train()
    optimizer.zero_grad()

    output = model(X_train)
    loss = criterion(output, y_train)
    loss.backward()
    optimizer.step()

    model.eval()
    with torch.no_grad():
        val_output = model(X_val)
        val_loss = criterion(val_output, y_val)
        _, predicted = torch.max(val_output, 1)
        val_accuracy = (predicted == y_val).float().mean()

    if (epoch+1) % 10 == 0:
        print(f'Epoch {epoch+1}, train Loss: {loss.item()}, val loss: {val_loss.item()}, val accuracy: {val_accuracy.item()}')
end = time.time()
print("Total time :", end-start)

/usr/local/lib/python3.10/dist-packages/torch/nn/modules/transformer.py:286: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")


Epoch 10, train Loss: 2.5165576934814453, val loss: 2.670081853866577, val accuracy: 0.2262156456708908
Epoch 20, train Loss: 2.3636014461517334, val loss: 2.5573790073394775, val accuracy: 0.22832980751991272
Epoch 30, train Loss: 2.2761974334716797, val loss: 2.5043275356292725, val accuracy: 0.23044396936893463
Epoch 40, train Loss: 2.225708484649658, val loss: 2.4864022731781006, val accuracy: 0.23678646981716156
Epoch 50, train Loss: 2.2060599327087402, val loss: 2.484729051589966, val accuracy: 0.23678646981716156
Epoch 60, train Loss: 2.1953582763671875, val loss: 2.486574172973633, val accuracy: 0.23044396936893463
Epoch 70, train Loss: 2.187142848968506, val loss: 2.493067502975464, val accuracy: 0.23678646981716156
Epoch 80, train Loss: 2.175198554992676, val loss: 2.505647659301758, val accuracy: 0.23467230796813965
Epoch 90, train Loss: 2.1708648204803467, val loss: 2.511640787124634, val accuracy: 0.23890063166618347
Epoch 100, train Loss: 2.1770477294921875, val loss: 2.5